In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm


In [2]:
dfPlaylists = pd.read_csv("./playlistData/spotify_dataset.csv", on_bad_lines='skip')

In [3]:
dfPlaylists.head()

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [4]:
print(len(dfPlaylists))

12891680


In [5]:
# loading playlist matrix
# takes the data frame to a matrix[playlistNumber][songNumber]

playlists = []
playlistNames = []
dfPlaylists.iloc[0][3]
for i in range(200000, 250000): # change to the whole thing at end
    currentPLName = dfPlaylists.iloc[i][3]
    if (currentPLName not in playlistNames):
        playlistNames.append(currentPLName)
        playlists.append([])
        
    playlists[playlistNames.index(currentPLName)].append((dfPlaylists.iloc[i][1], dfPlaylists.iloc[i][2]))

In [6]:
playlistsOG = playlists.copy()
print(len(playlists))

991


In [7]:
# single loop through all baskets
# count all single items
countSingeltons = {}

for i in playlists:
    for j in i:
        if (j not in countSingeltons):
            countSingeltons[j] = 1
        else:
            countSingeltons[j] += 1

In [8]:
# find all singeltons that are above the threshold

keys = countSingeltons.keys()
frequentSingeltons = {}
threshold = 5

for i in keys:
    if (countSingeltons[i] >= threshold):
        frequentSingeltons[i] = countSingeltons[i]

In [9]:
frequentSingeltonKeys = list(frequentSingeltons.keys())
print(frequentSingeltonKeys[0])

('Vampire Weekend', 'Cousins')


In [10]:
# If a playlist dosen't have and frequent singeltons,
# it wont have any frequent pairs, trim from search space
def trimPlaylistSingelton(playlists, frequentKeys):
    trimPlaylists = []
    for i in playlists:
        valid = False
        for j in i:
            if (j in frequentKeys):
                valid = True

        if (valid):
            trimPlaylists.append(i)
    
    return trimPlaylists

preLength = len(playlists)
playlists = trimPlaylistSingelton(playlists, frequentSingeltonKeys)
        
print(preLength, len(playlists))

991 354


In [11]:
#|       (0, 1) (0, 2) (0, 3) (0, 4) (0, 5)|     |   0  1  2  3  4 |
#|              (1, 2) (1, 3) (1, 4) (1, 5)|     |      5  6  7  8 |
#|                     (2, 3) (2, 4) (2, 5)|  =  |         9  10 11|
#|                            (3, 4) (3, 5)|     |            12 13|
#|                                   (4, 5)|     |               14|
#|                                         |     |                 |

In [12]:
# takes pair in matrix to triangular index
def matrixToTriangular(i, j, n):
    i = i + 1
    j = j + 1
    return int(((i - 1) * (n - (i / 2)) + j - i) - 1)

In [13]:
length = len(frequentSingeltonKeys)
maxIndex = matrixToTriangular(length - 2, length -1, length)
print(length, maxIndex)

666 221444


In [14]:
def countPairs(key1, key2, playlist):
    count = 0
    start = 0
    for i in playlist:
        if (key1 in i and key2 in i):
            count += 1

    return count

In [15]:
# triangular matrix to store the count of all possible pairs

triangularCounts1 = [0 for i in range(maxIndex + 1)]

def fillTriangleCounts(length, keys, playlist, triangularCounts):
    for i in range(0, length): 
        for j in range(i + 1, length):
            index = matrixToTriangular(i, j, length)
            triangularCounts[index] = countPairs(keys[i], keys[j], playlist)
            
fillTriangleCounts(length, frequentSingeltonKeys, playlists, triangularCounts1)


In [16]:
# looks through triangular count and find the ones above the threshold

frequentPairs = {}
lengthTriangular = len(triangularCounts1)
for i in range(length):
    for j in range(i + 1, length):
        index = matrixToTriangular(i, j, length)
        if (triangularCounts1[index] >= threshold):
            frequentPairs[(frequentSingeltonKeys[i], frequentSingeltonKeys[j])] = triangularCounts1[index]
            
print(len(list(frequentPairs.keys())[0]))
frequentPairKeys = list(frequentPairs.keys())

2


In [17]:
# trim playlists that dont have any frequent pairs
def trimPlaylistPairs(playlists, frequentKeys):
    trimPlaylists = []
    for i in playlists:
        isValidPlaylist = False
        for j in frequentKeys:
            isKeyItemsInPlaylist = [False for i in range(len(j))]
            for k in range(len(j)):
                if (j[k] in i):
                    isKeyItemsInPlaylist[k] = True
            if (False not in isKeyItemsInPlaylist):
                isValidPlaylist = True

        if (isValidPlaylist):
            trimPlaylists.append(i)
    
    return trimPlaylists

In [18]:
# generate possible triples based on what is in frequent pairs
possibleTriples = []

#frequentPairKeys = [("a", "b"), ("b", "c"), ("a", "c"), ("b", "d"), ("a", "d"), ("a", "e"), ("b", "e")]

# it can only be a triple if {a, b} {b, c} and {a, c} are all frequent doubles
for i in range(len(frequentPairKeys)):
    a = frequentPairKeys[i][0]
    b = frequentPairKeys[i][1]
    c = None
    
    doesExistBC = False
    doesExistAC = False
    for j in range(i + 1, len(frequentPairKeys)):
        if (frequentPairKeys[j][0] == a):
            c = frequentPairKeys[j][1]
            doesExistAC = True
        if (frequentPairKeys[j][1] == a):
            c = frequentPairKeys[j][0]
            doesExistAC = True
            
            
        if (frequentPairKeys[j][0] == b):
            c = frequentPairKeys[j][1]
            doesExistBC = True
        if (frequentPairKeys[j][1] == b):
            c = frequentPairKeys[j][0]
            doesExistBC = True
            
        if (c != None):
            frequentSublist = frequentPairKeys[j: len(frequentPairKeys)]
            if (doesExistBC):
                AC = (a, c) # AC is for some reason the only order that these show up in
                if (AC in frequentSublist):
                    doesExistAC = True
                
            if (doesExistAC):
                BC = (b, c)
                if (BC in frequentSublist):
                    doesExistBC = True
            
        if (doesExistBC and doesExistAC):
            possibleTriples.append((a, b, c))
        
        c = None
        doesExistBC = False
        doesExistAC = False
        
print(len(possibleTriples))

14973


In [19]:
preLength = len(playlists)
playlistsNew = trimPlaylistPairs(playlists, frequentPairKeys)
print(preLength, len(playlistsNew))

length = len(frequentPairKeys)
maxIndex = matrixToTriangular(length - 2, length -1, length)
print(length, maxIndex)

354 122
2640 3483479


In [20]:
print(len(frequentSingeltonKeys))
print(len(frequentPairKeys))
print(len(possibleTriples))

666
2640
14973


In [21]:
def countTriples(key1, key2, key3, playlist):
    count = 0

    for i in playlist:
        if (key1 in i and key2 in i and key3 in i):
            count += 1

    return count

In [22]:
# search through possible triples and find the ones that are actually frequent

frequentTriples = {}
for i in possibleTriples:
    count = countTriples(i[0], i[1], i[2], playlistsNew)
    if (count >= threshold):
        frequentTriples[i] = count
        
frequentTriplesKeys = list(frequentTriples.keys())
print(len(frequentTriplesKeys))
    

10316


In [23]:
possibleQuads = []

#frequentTriplesKeys = [("a", "b", "c"), ("a", "b", "d"), ("a", "c", "d"), ("b", "c", "d")]

# quad exists if {a, b, c}, {a, b, d}, {a, d, c}, and {d, b, c} exist
for i in range(len(frequentTriplesKeys)):
    a = frequentTriplesKeys[i][0]
    b = frequentTriplesKeys[i][1]
    c = frequentTriplesKeys[i][2]
    d = None
    
    doesExistABD = False
    doesExistADC = False
    doesExistBDC = False
    for j in range(i + 1, len(frequentTriplesKeys)):
        # Located {a, b, d}
        if (a in frequentTriplesKeys[j] and b in frequentTriplesKeys[j]):
            indexA, indexB = frequentTriplesKeys[j].index(a), frequentTriplesKeys[j].index(b)
            indexD = 3 - (indexA + indexB)
            d = frequentTriplesKeys[j][indexD]
            doesExistABD = True

        # Located {a, d, c}
        if (a in frequentTriplesKeys[j] and c in frequentTriplesKeys[j]):
            indexA, indexC = frequentTriplesKeys[j].index(a), frequentTriplesKeys[j].index(c)
            indexD = 3 - (indexA + indexC)
            d = frequentTriplesKeys[j][indexD]
            doesExistADC = True
            
        # Located {d, b, c}
        if (b in frequentTriplesKeys[j] and c in frequentTriplesKeys[j]):
            indexB, indexC = frequentTriplesKeys[j].index(b), frequentTriplesKeys[j].index(c)
            indexD = 3 - (indexC + indexB)
            d = frequentTriplesKeys[j][indexD]
            doesExistBDC = True
            
        if (d != None):
            frequentSublist = frequentTriplesKeys[j: len(frequentTriplesKeys)]
            # Looking for {a, d, c} {a, c, d} {d, a, c} {d, c, a} {c, a, d} {c, d, a} and 
            #             {b, d, c} {b, c, d} {c, b, d} {c, d, b} {d, c, b} {d, b, c}
            if (doesExistABD):
                ACD = (a, c, d) # the only way for {a, d, c} to appear is in order ACD (because of how the keys are stored)
                BCD = (b, c, d)
                if (ACD in frequentSublist):
                    doesExistADC = True
                if (BCD in frequentSublist):
                    doesExistBDC = True

            if (doesExistADC):
                ABD = (a, b, d)
                BCD = (b, c, d)
                
                if (ABD in frequentSublist):
                    doesExistABD = True
                if (BCD in frequentSublist):
                    doesExistBDC = True
                    
            if (doesExistBDC):
                ADC = (a, c, d)
                ABD = (a, b, d)
                
                if (ADC in frequentSublist):
                    doesExistADC = True
                if (ABD in frequentSublist):
                    doesExistABD = True
                
                
        if (doesExistABD and doesExistADC and doesExistBDC):
            possibleQuads.append((a, b, c, d))
        
        d = None
        doesExistABD = False
        doesExistADC = False
        doesExistBDC = False
        
print(len(possibleQuads))

31772


In [24]:
def countQuads(key1, key2, key3, key4, playlist):
    count = 0

    for i in playlist:
        if (key1 in i and key2 in i and key3 in i and key4 in i):
            count += 1

    return count

In [25]:
frequentQuads = {}
for i in possibleQuads:
    count = countQuads(i[0], i[1], i[2], i[3], playlistsNew)
    if (count >= threshold):
        frequentQuads[i] = count
        
frequentQuadsKeys = list(frequentQuads.keys())
print(len(frequentQuadsKeys))

31594


In [26]:
# if (1, 2, 3, 4) is a frequent quad and (1, 2, 3) is a frequent triple, only (1, 2, 3, 4) is maximal
# copy frequent dictionaries and remove elements that are not maximal

maximalQuads = dict(frequentQuads)
maximalTriples = frequentTriplesKeys.copy()
maximalPairs = frequentPairKeys.copy()

print(len(maximalQuads))
print(len(maximalTriples))
for i in frequentTriplesKeys:
    a = i[0]
    b = i[1]
    c = i[2]
    
    for j in frequentQuadsKeys:
        if (a in j and b in j and c in j):
            index = maximalTriples.index(i)
            maximalTriples.pop(index)
            break
        
print(len(maximalTriples))

print(len(maximalPairs))
for i in frequentPairKeys:
    a = i[0]
    b = i[1]
    
    for j in frequentQuadsKeys:
        if (a in j and b in j):
            index = maximalPairs.index(i)
            maximalPairs.pop(index)
            break

print(len(maximalPairs))
maxPairsIntermediate = maximalPairs.copy()
for i in maxPairsIntermediate:
    a = i[0]
    b = i[1]
    for j in maximalTriples:
        if (a in j and b in j):
            index = maximalPairs.index(i)
            maximalPairs.pop(index)
            break
    
print(len(maximalPairs))

31594
10316
112
2640
346
176


In [27]:
# load maximal matrix with values from maximal pairs, triples, and quads

maximal = maximalQuads.copy()
for i in maximalTriples:
    maximal[i] = frequentTriples[i]
    
    print(frequentTriples[i])
    
for i in maximalPairs:
    maximal[i] = frequentPairs[i]
    
maximalMatrix = []
for i in maximal.keys():
    vectorMaximal = []
    if (len(i) == 4):
        vectorMaximal.append(i[0])
        vectorMaximal.append(i[1])
        vectorMaximal.append(i[2])
        vectorMaximal.append(i[3])
    
    if (len(i) == 3):
        vectorMaximal.append(i[0])
        vectorMaximal.append(i[1])
        vectorMaximal.append(i[2])
        vectorMaximal.append(None)
    
    if (len(i) == 2):
        vectorMaximal.append(i[0])
        vectorMaximal.append(i[1])
        vectorMaximal.append(None)
        vectorMaximal.append(None)
        
    maximalMatrix.append(vectorMaximal)

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [28]:
# These are all the valid association rules for a quad maximal
# The maximal (1, 2, 3, 4) implies rules for both (1, 2, 3) -> 4 and (1) -> 4

# (1, 2, 3, 4) = {1: (2, 3, 4) -> 1}, {2: (1, 3, 4) -> 2}, {3: (1, 2, 4) -> 3}, {4: (1, 2, 3) -> 4}, 
#      (1, 2, 3) { 5: (2, 3) -> 1}, { 6: (1, 3) -> 2}, { 7: (1, 2) -> 3},
#      (1, 2, 4) { 8: (2, 4) -> 1}, { 9: (1, 4) -> 2}, {10: (1, 2) -> 4},
#      (1, 3, 4) {11: (3, 4) -> 1}, {12: (1, 4) -> 3}, {13: (1, 3) -> 4},
#      (2, 3, 4) {14: (3, 4) -> 2}, {15: (2, 4) -> 3}, {16: (2, 3) -> 4},
#             (1, 2) {17: (2) -> 1}, {18: (1) -> 2},
#             (1, 3) {19: (3) -> 1}, {20: (1) -> 3},
#             (1, 4) {21: (4) -> 1}, {22: (1) -> 4},
#             (2, 3) {23: (3) -> 2}, {24: (2) -> 3},
#             (2, 4) {25: (4) -> 2}, {26: (2) -> 4},
#             (3, 4) {27: (4) -> 3}, {28: (3) -> 4}

# (1, 2, 3) = {1: (2, 3) -> 1}, {2: (1, 3) -> 2}, {3: (1, 2) -> 3}
#      (1, 2) {4: (2) -> 1}, {5: (1) -> 2},
#      (1, 3) {6: (3) -> 1}, {7: (1) -> 3},
#      (2, 3) {8: (3) -> 2}, {9: (2) -> 3}

# (1, 2) = {1: (2) -> 1}, {2: (1) -> 2}

# calculating intrest for all association rules
intrestMatrix = []
count = 0
for i in maximalMatrix:
    # because it's packaged as one matrix, there are some nan values
    intrestMaximal = []
    i = maximalMatrix[count].copy()
    if (None in i):
        i.remove(None)
        if (None in i):
            i.remove(None)
            
    # quad
    if (len(i) == 4):
        # (1, 2, 3) -> 4   {from quads}
        countFull = frequentQuads[tuple(i)]
        for j in i:
            subsection = i.copy()
            subsection.remove(j)
            countj = countSingeltons[j]
            countSubsection = frequentTriples[tuple(subsection)]
            confidence = countFull / countSubsection
            support = countj / len(playlists)
            intrest = confidence - support
            intrestMaximal.append(intrest)
            
        # (1, 2) -> 3   {from quads}
        for k in range(4):
            if (k == 0):
                sub1 = [i[0], i[1], i[2]]
            elif (k == 1):
                sub1 = [i[0], i[1], i[3]]
            elif (k == 2):
                sub1 = [i[0], i[2], i[3]]
            elif (k == 3):
                sub1 = [i[1], i[2], i[3]]
                
            countFull = frequentTriples[tuple(sub1)]
            for j in sub1:
                subsection = sub1.copy()
                subsection.remove(j)
                countj = countSingeltons[j]
                countSubsection = frequentPairs[tuple(subsection)]
                confidence = countFull / countSubsection
                support = countj / len(playlists)
                intrest = confidence - support
                intrestMaximal.append(intrest)
                
        # (1) -> 2   {from quads}
        for k in range(6):
            if (k == 0):
                sub2 = [i[0], i[1]]
            elif (k == 1):
                sub2 = [i[0], i[2]]
            elif (k == 2):
                sub2 = [i[0], i[3]]
            elif (k == 3):
                sub2 = [i[1], i[2]]
            elif (k == 4):
                sub2 = [i[1], i[3]]
            elif (k == 5):
                sub2 = [i[2], i[3]]
                
            countFull = frequentPairs[tuple(sub2)]
            for j in sub2:
                subsection = sub2.copy()
                subsection.remove(j)
                subsection = subsection[0]
                countj = countSingeltons[j]
                countSubsection = frequentSingeltons[tuple(subsection)]
                confidence = countFull / countSubsection
                support = countj / len(playlists)
                intrest = confidence - support
                intrestMaximal.append(intrest)
                
            
    # triples
    elif (len(i) == 3):
        # (1, 2) -> 3   {from triples}
        countFull = frequentTriples[tuple(i[:3])]
        for j in i:
            subsection = i.copy()
            subsection.remove(j)
            countj = countSingeltons[j]
            countSubsection = frequentPairs[tuple(subsection)]
            confidence = countFull / countSubsection
            support = countj / len(playlists)
            intrest = confidence - support
            intrestMaximal.append(intrest)
        
        # (1) -> 2   {from triples}
        for k in range(3):
            if (k == 0):
                sub2 = [i[0], i[1]]
            elif (k == 1):
                sub2 = [i[0], i[2]]
            elif (k == 2):
                sub2 = [i[1], i[2]]

            countFull = frequentPairs[tuple(sub2)]
            for j in sub2:
                subsection = sub2.copy()
                subsection.remove(j)
                subsection = subsection[0]
                countj = countSingeltons[j]
                countSubsection = frequentSingeltons[tuple(subsection)]
                confidence = countFull / countSubsection
                support = countj / len(playlists)
                intrest = confidence - support
                intrestMaximal.append(intrest)
                
        # padding to make the matrix square
        # (if i wanted to get rid of this, i could pass each one as a seperate matrix, 
        # but there are not that many maximals that are not quads, so it isn't that much space)
        for j in range(19):
            intrestMaximal.append(None)
        
    # pairs
    elif(len(i) == 2):
        # (1) -> 2   {from pairs}
        countFull = frequentPairs[tuple(i[:3])]
        for j in i:
            subsection = i.copy()
            subsection.remove(j)
            subsection = subsection[0]
            countj = countSingeltons[j]
            countSubsection = frequentSingeltons[tuple(subsection)]
            confidence = countFull / countSubsection
            support = countj / len(playlists)
            intrest = confidence - support
            intrestMaximal.append(intrest)
        
        for j in range(24):
            intrestMaximal.append(None)
    
    intrestMatrix.append(intrestMaximal)
    count +=1

In [29]:
for i in range(len(intrestMatrix)):
    for j in intrestMatrix[i]:
        maximalMatrix[i].append(j)

In [30]:
maximalsDF = pd.DataFrame(maximalMatrix)
maximalsDF.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,"(Ryan Adams, Am I Safe)","(Ryan Adams, Gimme Something Good)","(R.E.M., Imitation Of Life)","(R.E.M., Man On The Moon)",0.819209,0.980226,0.966102,0.971751,0.819209,0.980226,...,0.402542,0.966102,0.485876,0.971751,0.480226,0.823245,0.580226,0.828894,0.766102,0.638418
1,"(The Head And The Heart, Another Story)","(Damien Jurado, Arkansas)","(Matt Pond PA, Brooklyn Stars)","(The War On Drugs, Eyes To The Wind)",0.805085,0.816384,0.968927,0.980226,0.805085,0.608051,...,0.699024,0.768927,0.828894,0.580226,0.528505,0.968927,0.840194,0.980226,0.968927,0.616590
2,"(The Head And The Heart, Another Story)","(Damien Jurado, Arkansas)","(Matt Pond PA, Brooklyn Stars)","(Blitzen Trapper, Furr)",0.805085,0.983051,0.968927,0.983051,0.805085,0.608051,...,0.699024,0.768927,0.805085,0.483051,0.528505,0.968927,0.983051,0.983051,0.968927,0.528505
3,"(The Head And The Heart, Another Story)","(Damien Jurado, Arkansas)","(Matt Pond PA, Brooklyn Stars)","(Jakob Dylan, Holy Rollers For Love)",0.971751,0.983051,0.968927,0.985876,0.805085,0.608051,...,0.699024,0.768927,0.971751,0.485876,0.528505,0.968927,0.983051,0.819209,0.968927,0.440421
4,"(The Head And The Heart, Another Story)","(Damien Jurado, Arkansas)","(Matt Pond PA, Brooklyn Stars)","(R.E.M., Man On The Moon)",0.805085,0.816384,0.968927,0.971751,0.805085,0.608051,...,0.699024,0.768927,0.771751,0.771751,0.528505,0.968927,0.583051,0.971751,0.668927,0.608115


In [31]:
# export to csv
maximalsDF.to_csv("./playlistData/maximal5_T5_50.csv")